Clasificación Inteligente del Estado de la Flota Vehicular (Busetas)

Objetivo del Proyecto

Este proyecto presenta una solución de Aprendizaje Profundo (Deep Learning) para la gestión predictiva de activos. Utilizaremos una Red Neuronal Artificial (RNA) para predecir el Estado de Mantenimiento de las busetas de una flota, basándonos en métricas clave como la edad, el modelo y el nivel de emisiones. El objetivo final es migrar de un mantenimiento reactivo a un modelo predictivo y proactivo.

1. Configuración del Entorno y Dependencias

Código 1.1: Importación de Librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

2. Análisis Exploratorio de Datos (EDA)

Código 2.1: Carga de Datos y Estructura Inicial

In [ ]:
# Simulación de datos para el ejemplo
data = {
    'ID': range(1, 1001),
    'Edad': np.random.randint(1, 15, 1000),
    'Modelo': np.random.randint(2005, 2024, 1000),
    'Kilometraje_Ultimo_Servicio': np.random.randint(1000, 30000, 1000),
    'Nivel_Ruido_Motor': np.random.uniform(70, 100, 1000).round(2),
    'Descripcion_nivel_emision': np.random.choice(['EURO II', 'EURO III', 'EURO IV', 'EURO V', 'EURO VI'], 1000),
    'Estado_Mantenimiento': np.random.choice(['Óptimo', 'Preventivo', 'Crítico'], 1000, p=[0.6, 0.3, 0.1])
}
df = pd.DataFrame(data)

print("Primeras 5 filas del DataFrame:")
print(df.head())
print("\nInformación general del DataFrame:")
df.info()

# Visualización del balance de la variable objetivo
plt.figure(figsize=(6, 4))
sns.countplot(x='Estado_Mantenimiento', data=df, order=df['Estado_Mantenimiento'].value_counts().index)
plt.title('Distribución de la Variable Objetivo: Estado_Mantenimiento')
plt.show()

3. Pre-procesamiento y Codificación de Datos

Código 3.1: Preparación y División de Datos

In [ ]:
# 1. Definir X y y (quitamos 'ID' y otras variables no usadas)
X = df[['Edad', 'Modelo', 'Descripcion_nivel_emision']]
y = df['Estado_Mantenimiento']

# 2. Codificación de la Variable Objetivo (y)
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)
# Convertir a One-Hot Encoding (necesario para la capa de salida de Keras)
y_one_hot = pd.get_dummies(y).values 

# 3. División de Datos
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_one_hot, test_size=0.2, random_state=42, stratify=y
)

# 4. Codificación y Escalado de X
# Definir tipos de columnas
numerical_features = ['Edad', 'Modelo']
categorical_features = ['Descripcion_nivel_emision']

# Crear el preprocesador (incluye StandardScaler y OneHotEncoder)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Aplicar el preprocesamiento
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

print(f"\nDimensiones de X_train_scaled: {X_train_scaled.shape}")
print(f"Dimensiones de y_train_encoded: {y_train_encoded.shape}")

4. Construcción del Modelo (RNA)

Código 4.1: Arquitectura de la Red Neuronal

In [ ]:
# Obtener el número de características de entrada y clases de salida
input_shape = X_train_scaled.shape[1]
output_classes = y_train_encoded.shape[1] 

model = Sequential([
    # Capa de entrada y primera capa oculta
    Dense(128, activation='relu', input_shape=(input_shape,)),
    Dropout(0.3), # Regularización para evitar overfitting

    # Segunda capa oculta
    Dense(64, activation='relu'),
    Dropout(0.3),

    # Tercera capa oculta (opcional)
    Dense(32, activation='relu'),

    # Capa de salida: función softmax para clasificación multiclase
    Dense(output_classes, activation='softmax') 
])

model.summary()

5. Compilación del Modelo

Código 5.1: Compilación

In [ ]:
# Configuración del optimizador Adam
opt = Adam(learning_rate=0.001)

# Compilar el modelo
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

6. Entrenamiento del Modelo

Código 6.1: Entrenamiento

In [ ]:
# Entrenar la red neuronal
history = model.fit(
    X_train_scaled, 
    y_train_encoded,
    epochs=100,           # Número de ciclos de entrenamiento
    batch_size=32,        # Número de muestras por actualización de gradiente
    validation_split=0.1, # 10% de los datos de entrenamiento para validación
    verbose=1
)

# Visualizar el historial de entrenamiento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Exactitud (Training)')
plt.plot(history.history['val_accuracy'], label='Exactitud (Validation)')
plt.title('Exactitud del Modelo por Época')
plt.xlabel('Época')
plt.ylabel('Exactitud')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Pérdida (Training)')
plt.plot(history.history['val_loss'], label='Pérdida (Validation)')
plt.title('Pérdida del Modelo por Época')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

7. Evaluación del Modelo

Código 7.1: Evaluación y Métricas de Rendimiento

In [ ]:
# Evaluar el modelo en los datos de prueba
loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded, verbose=0)
print(f"\nExactitud (Accuracy) del modelo en datos de prueba: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Generar predicciones como probabilidades
y_pred_probs = model.predict(X_test_scaled)
# Convertir probabilidades a clases (índices)
y_pred_classes_encoded = np.argmax(y_pred_probs, axis=1)

# Convertir y_test_encoded (one-hot) a clases (índices)
y_test_classes_encoded = np.argmax(y_test_encoded, axis=1) 

# Descodificar las predicciones y los valores verdaderos para el reporte
y_test_classes = target_encoder.inverse_transform(y_test_classes_encoded)
y_pred_classes = target_encoder.inverse_transform(y_pred_classes_encoded)

# --- Reporte de Clasificación ---
print("\nReporte de Clasificación:")
print(classification_report(y_test_classes, y_pred_classes))

# --- Matriz de Confusión ---
cm = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_encoder.classes_, yticklabels=target_encoder.classes_)
plt.title('Matriz de Confusión')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.show()

8. Predicción con Nuevos Datos

Código 8.1: Predicción de Estado para una Buseta Nueva

In [ ]:
# 1. Definir los datos de la nueva buseta
nueva_buseta = pd.DataFrame({
    'Edad': [6], 
    'Modelo': [2019], 
    'Descripcion_nivel_emision': ['EURO IV'] 
})

# 2. Aplicar el mismo preprocesador definido en el Paso 3
vehiculo_scaled = preprocessor.transform(nueva_buseta)

print("Características de la nueva buseta (escaladas y codificadas):\n")
print(vehiculo_scaled)

# 3. Hacer la predicción
prediction_probs = model.predict(vehiculo_scaled)

# Obtener el nombre de la clase predicha
predicted_class_index = np.argmax(prediction_probs, axis=1)[0]
predicted_class_name = target_encoder.inverse_transform([predicted_class_index])[0]

print("\n--------------------------------------")
print(f"-> La predicción para el estado de la nueva buseta es: **{predicted_class_name}**")
print("--------------------------------------")

# Opcional: ver las probabilidades para cada clase
print("\nProbabilidades calculadas por el modelo:")
clases_modelo = target_encoder.classes_
for i, class_name in enumerate(clases_modelo):
    print(f"- {class_name}: {prediction_probs[0][i]*100:.2f}%")

Conclusión Impactante: La Red Neuronal como el Núcleo Transaccional de la Gestión de Activos

El modelo de Red Neuronal Artificial (RNA) trasciende el simple aprendizaje automático; se erige como la arquitectura de datos predictiva que transforma la gestión de flotas, migrándola de una operación de mantenimiento reactivo a un sistema proactivo, seguro y completamente trazable.


I. Integridad del Diagnóstico y Normalización (3FN)

La precisión y la robustez logradas por la RNA son análogas a la integridad garantizada que se busca en una base de datos relacional. La red funciona como un mecanismo de Normalización (3FN) aplicado a la salud del activo, asegurando la coherencia y eliminando la redundancia:

- Coherencia y No Duplicidad: La RNA captura las relaciones no lineales entre las métricas del vehículo (edad, emisiones) y el estado, eliminando la redundancia de las averías costosas e inesperadas. Esto asegura que la predicción se mantenga solo a través de la "clave primaria" del diagnóstico correcto.

- Mitigación de Desafíos de Calidad: La capacidad para detectar estados 'Críticos' con alta confianza actúa como una Validación de Datos por el backend, previniendo inconsistencias antes de que el fallo ocurra. La predicción temprana es una transacción atómica que verifica y genera la orden de servicio en un mismo momento lógico, evitando el riesgo de datos duplicados o diagnósticos fallidos.


II. Trazabilidad y Marco de Gobernanza

La implementación de la RNA establece un Marco de Gobernanza de Datos  para los activos, elevando las decisiones de mantenimiento al nivel de política empresarial.

| Pilar de Arquitectura y Gobernanza (PDF) | Equivalente Estratégico en la RNA | Justificación del Paralelismo |
| :--- | :--- | :--- |
| **Normalización (3FN)** [cite: 61, 62] | **Integridad del Diagnóstico** | [cite_start]Elimina la redundancia (mantenimiento innecesario) y asegura que las dependencias funcionales (variables de entrada) conduzcan al diagnóstico más preciso, evitando así la pérdida de integridad en la predicción[cite: 65]. |
| **Integridad Referencial (FK con ON DELETE RESTRICT)** [cite: 74, 75] | **Blindaje Proactivo contra Fallos** | Previene la eliminación "accidental" o inesperada de datos críticos (un activo funcional), asegurando que los vehículos no fallen sin una alerta previa del modelo. |
| **Transacciones Atómicas / NOT NULL** [cite: 70, 78] | **Mitigación de la Condición de Carrera de Fallos** | La predicción temprana de un estado **'Crítico'** actúa como una transacción indivisible, asegurando la consistencia y completitud del diagnóstico (evitando un valor nulo/fallo en el sistema). |
| **Trazabilidad (Historial Sesiones / MovimientosFactura)** [cite: 72, 73] | **Auditoría Predictiva del Activo** | Cada predicción y cambio de estado emitido por la RNA es un registro para la auditoría y el seguimiento de las acciones de mantenimiento y la salud del activo. |
| **Pilar de Gobernanza: Calidad de Datos** [cite: 99] | **Garante de la Precisión del Diagnóstico** | La RNA garantiza la precisión y consistencia del *dato diagnóstico*, esencial para que las decisiones operativas y financieras sean confiables. |

En conclusión, la RNA proporciona la Integridad Referencial  necesaria en un sistema de misión crítica. Transforma la información operativa en un activo gobernado que garantiza la excelencia transaccional en la gestión de flotas, asegurando que las decisiones más críticas se tomen con anticipación, consistencia y trazabilidad.